# E-commerce Return Risk – Google Colab Notebook

This notebook lets you:
1) Upload `orders.csv` and `returns.csv`
2) Clean & merge data, create `Returned` flag
3) Run exploratory analysis (return rates by product & seller)
4) Train a Logistic Regression model
5) Export **high_risk_products.csv** and **item_return_risk_scores.csv** for Power BI

**Tip:** Run each cell in order (Shift + Enter).

In [ ]:
# STEP 1: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report
import joblib
from datetime import datetime
print('Libraries imported.')

In [ ]:
# STEP 2: Upload CSV files (orders.csv and returns.csv)
from google.colab import files
print('Please upload orders.csv and returns.csv')
uploaded = files.upload()  # Choose both files

assert 'orders.csv' in uploaded, 'orders.csv was not uploaded.'
assert 'returns.csv' in uploaded, 'returns.csv was not uploaded.'

orders = pd.read_csv('orders.csv')
returns = pd.read_csv('returns.csv')
print('Files loaded:', orders.shape, returns.shape)
orders.head()

In [ ]:
# STEP 3: Basic validation
print('Orders columns:', list(orders.columns))
print('Returns columns:', list(returns.columns))

# Expected minimal columns
expected_orders = {'Order Id','Order Date','Delivery Date','Buyer Id','Buyer Name','Product Id','Product Name','Seller Id','Seller Name'}
expected_returns = {'Order Id','Return Date','Product Id','Product Name','Seller Id','Seller Name'}

missing_o = expected_orders - set(orders.columns)
missing_r = expected_returns - set(returns.columns)
if missing_o:
    print('WARNING: orders.csv missing columns:', missing_o)
if missing_r:
    print('WARNING: returns.csv missing columns:', missing_r)

orders.isna().sum().sort_values(ascending=False).head(10)

In [ ]:
# STEP 4: Clean & merge – create Returned flag at (Order Id, Product Id) level
orders['Order Date'] = pd.to_datetime(orders['Order Date'], errors='coerce')
orders['Delivery Date'] = pd.to_datetime(orders['Delivery Date'], errors='coerce')
if 'Return Date' in returns.columns:
    returns['Return Date'] = pd.to_datetime(returns['Return Date'], errors='coerce')

# Deduplicate returns at line-item level, keep latest return per (Order Id, Product Id)
returns_sorted = returns.sort_values(['Order Id','Product Id','Return Date']).drop_duplicates(['Order Id','Product Id'], keep='last')

# Merge on Order Id + Product Id
merged = orders.merge(
    returns_sorted[['Order Id','Product Id','Return Date']],
    on=['Order Id','Product Id'], how='left', validate='m:1'
)
merged['Returned'] = merged['Return Date'].notna().astype(int)

# Feature: delivery days
merged['Delivery_Days'] = (merged['Delivery Date'] - merged['Order Date']).dt.days

print('Rows:', len(merged), '| Return rate:', round(merged['Returned'].mean(), 4))
merged.head()

In [ ]:
# STEP 5: Exploratory analysis
overall_rr = merged['Returned'].mean()
print('Overall Return Rate:', round(overall_rr, 4))

# Top 10 products by return rate (min volume filter)
prod_stats = merged.groupby(['Product Id','Product Name']).agg(
    items=('Returned','size'), rr=('Returned','mean')
).reset_index()
prod_stats = prod_stats[prod_stats['items'] >= 5]  # filter small volume
top_products = prod_stats.sort_values('rr', ascending=False).head(10)
print(top_products)

# Plot top sellers by return rate (min 10 items)
seller_stats = merged.groupby(['Seller Id','Seller Name']).agg(
    items=('Returned','size'), rr=('Returned','mean')
).reset_index()
seller_stats = seller_stats[seller_stats['items'] >= 10].sort_values('rr', ascending=False).head(10)
ax = seller_stats.plot(kind='bar', x='Seller Name', y='rr', title='Top Sellers by Return Rate (min 10 items)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# STEP 6: Model – Logistic Regression (One-Hot for product & seller, plus delivery days)
from sklearn.utils.class_weight import compute_class_weight

features = ['Product Name','Seller Name','Delivery_Days']
X = merged[features].copy()
y = merged['Returned'].astype(int)

cat_cols = ['Product Name','Seller Name']
num_cols = ['Delivery_Days']

preprocess = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
pipe = Pipeline([
    ('prep', preprocess),
    ('clf', LogisticRegression(max_iter=300, class_weight='balanced'))
])
pipe.fit(X_train, y_train)

proba_test = pipe.predict_proba(X_test)[:,1]
roc = roc_auc_score(y_test, proba_test)
ap = average_precision_score(y_test, proba_test)
print({'ROC_AUC': round(roc,4), 'PR_AUC': round(ap,4)})
print(classification_report(y_test, (proba_test>=0.5).astype(int)))

# Save model (optional)
joblib.dump(pipe, 'return_logreg.pkl')
print('Model saved: return_logreg.pkl')

In [ ]:
# STEP 7: Score all items & export CSVs
scores = pipe.predict_proba(merged[features])[:,1]
merged['risk_score'] = scores
merged['risk_bucket'] = pd.qcut(merged['risk_score'], q=[0, 0.8, 0.95, 1.0], labels=['Low','Medium','High'])

# Export item-level scores
item_path = 'item_return_risk_scores.csv'
merged.to_csv(item_path, index=False)
print('Saved', item_path)

# Product-level high risk summary
prod = merged.groupby(['Product Id','Product Name']).agg(
    avg_risk=('risk_score','mean'),
    items=('risk_score','size'),
    high_share=('risk_bucket', lambda s: (s=='High').mean())
).reset_index().sort_values(['avg_risk','items'], ascending=False)
prod['is_high_risk'] = ((prod['avg_risk']>=0.6) | (prod['high_share']>=0.3)).astype(int)
prod_path = 'high_risk_products.csv'
prod.to_csv(prod_path, index=False)
print('Saved', prod_path)

# Download files
from google.colab import files
files.download(item_path)
files.download(prod_path)

## Next Steps (Optional)
- Add more features if available (price, discount, channel, geography)
- Create a Power BI dashboard using the exported CSVs
- Try time-based splits (train on earlier months, test on recent months)